<a href="https://colab.research.google.com/github/dataSci-rigo/hum0rB0T3000/blob/master/Hum0B0t3000_BertModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# BERT End to End (Fine-tuning + Predicting) in 5 minutes with Cloud TPU

## Overview

**BERT**, or **B**idirectional **E**mbedding **R**epresentations from **T**ransformers, is a new method of pre-training language representations which obtains state-of-the-art results on a wide array of Natural Language Processing (NLP) tasks. The academic paper can be found here: https://arxiv.org/abs/1810.04805.

This Colab demonstates using a free Colab Cloud TPU to fine-tune sentence and sentence-pair classification tasks built on top of pretrained BERT models and 
run predictions on tuned model. The colab demonsrates loading pretrained BERT models from both [TF Hub](https://www.tensorflow.org/hub) and checkpoints.

**Note:**  You will need a GCP (Google Compute Engine) account and a GCS (Google Cloud 
Storage) bucket for this Colab to run.

Please follow the [Google Cloud TPU quickstart](https://cloud.google.com/tpu/docs/quickstart) for how to create GCP account and GCS bucket. You have [$300 free credit](https://cloud.google.com/free/) to get started with any GCP product. You can learn more about Cloud TPU at https://cloud.google.com/tpu/docs.

This notebook is hosted on GitHub. To view it in its original repository, after opening the notebook, select **File > View on GitHub**.

## Learning objectives

In this notebook, you will learn how to train and evaluate a BERT model using TPU.

## Instructions

<h3><a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a>  &nbsp;&nbsp;Train on TPU</h3>

   1. Create a Cloud Storage bucket for your TensorBoard logs at http://console.cloud.google.com/storage and fill in the BUCKET parameter in the "Parameters" section below.
 
   1. On the main menu, click Runtime and select **Change runtime type**. Set "TPU" as the hardware accelerator.
   1. Click Runtime again and select **Runtime > Run All** (Watch out: the "Colab-only auth for this notebook and the TPU" cell requires user input). You can also run the cells manually with Shift-ENTER.

### Set up your TPU environment

In this section, you perform the following tasks:

*   Set up a Colab TPU running environment
*   Verify that you are connected to a TPU device
*   Upload your credentials to TPU to access your GCS bucket.

In [4]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.
import pandas as pd


TPU address is grpc://10.6.80.130:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 17864271032720771166),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 8628967968841390208),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 2631560272620451248),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 4452277740090177287),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 4235196487869766127),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 4432635597286159955),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 9120617361511650660),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 5462997468622014564),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 1503975175189360

In [0]:
reddit = pd.read_json('https://query.data.world/s/wtye65og2jr3slglflm5m2q6weu2kg')
stupid_stuff = pd.read_json('https://query.data.world/s/3zcjma2uidkf6tzt6olm2o2cxf75x5')
wocka = pd.read_json('https://query.data.world/s/4bamgqs4m5i7mlnz4la5wljzpxnok5')

### Prepare and import BERT modules
​
With your environment configured, you can now prepare and import the BERT modules. The following step clones the source code from GitHub and import the modules from the source. Alternatively, you can install BERT using pip (!pip install bert-tensorflow).

In [0]:
import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

# import python modules defined by BERT
import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization
import numpy as np

# import tfhub 
import tensorflow_hub as hub

### Prepare for training

This next section of code performs the following tasks:

*  Specify task and download training data.
*  Specify BERT pretrained model
*  Specify GS bucket, create output directory for model checkpoints and eval results.




In [7]:
TASK = 'Jokes' #@param {type:"string"}
assert TASK in ('MRPC', 'CoLA',"Jokes"), 'Only (MRPC, CoLA) are demonstrated here.'

if TASK != "Jokes":

  # Download glue data.
  ! test -d download_glue_repo || git clone https://gist.github.com/60c2bdb54d156a41194446737ce03e2e.git download_glue_repo
  !python download_glue_repo/download_glue_data.py --data_dir='glue_data' --tasks=$TASK

  TASK_DATA_DIR = 'glue_data/' + TASK
  print('***** Task data directory: {} *****'.format(TASK_DATA_DIR))
  !ls $TASK_DATA_DIR

BUCKET = 'humorbot3000' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'
OUTPUT_DIR = 'gs://{}/hum0rb0t/3000/{}'.format(BUCKET, TASK)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
BERT_MODEL = 'uncased_L-24_H-1024_A-16' #@param {type:"string"}
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'

***** Model output directory: gs://humorbot3000/hum0rb0t/3000/Jokes *****


Now let's load tokenizer module from TF Hub and play with it.

In [8]:
tokenizer = run_classifier_with_tfhub.create_tokenizer_from_hub_module(BERT_MODEL_HUB)
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

In [13]:
print(reddit.shape)
print(reddit.columns)
print(reddit['title'].head(5))
print(reddit['body'].head(5))
print(reddit['score'].max())
print(reddit['score'].mean())
print(reddit['score'].median())
print(len(reddit['score']))
print(np.unique(reddit['score'],return_counts=True))
print(len(reddit[reddit['score']>10]))
print(len(reddit[reddit['score']>15])/len(reddit['score']))



(194553, 4)
Index(['body', 'id', 'score', 'title'], dtype='object')
0     I hate how you cant even say black paint anymore
1    What's the difference between a Jew in Nazi Ge...
2                       I recently went to America....
3    Brian raises his hand and says, “He’s in Heaven.”
4    You hear about the University book store worke...
Name: title, dtype: object
0    Now I have to say "Leroy can you please paint ...
1    Pizza doesn't scream when you put it in the ov...
2    ...and being there really helped me learn abou...
3    A Sunday school teacher is concerned that his ...
4    He got caught trying to sell the two books to ...
Name: body, dtype: object
48526
118.22325535972203
3.0
194553
(array([    0,     1,     2, ..., 39570, 45500, 48526]), array([61561, 19347, 13684, ...,     1,     1,     1]))
58927
0.2540438852137978


In [0]:
red=reddit.copy(deep=True)
reddit['score']=pd.np.digitize( reddit.score, bins = [15])

In [15]:
pos=reddit[reddit['score']==1]
neg=reddit[reddit['score']==0]
neg=neg.sample(frac=0.333,random_state=200)
print(len(neg),len(pos))

47806 50990


In [16]:
#DataFrame.append(self, other, ignore_index=False, verify_integrity=False, sort=False) → 'DataFrame

reddit=neg.append(pos,ignore_index=True,sort=True)
np.unique(reddit['score'],return_counts=True)

(array([0, 1]), array([47806, 50990]))

In [0]:
class JokeProcessor(run_classifier.DataProcessor):
  """Processor for the Quora Question pair data set."""

  def get_train_examples(self, data_dir):
    """Reading train.tsv and converting to list of InputExample"""
    return self._create_examples(data_dir[0], 'train')

  def get_dev_examples(self, data_dir):
    """Reading dev.tsv and converting to list of InputExample"""
    return self._create_examples(data_dir[1], 'dev')
  
  def get_test_examples(self, data_dir):
    """Reading train.tsv and converting to list of InputExample"""
    return self._create_examples(
        self._read_tsv(data_dir+'test.tsv'), 'test')
  
  def convert_examples(self, sentence_pairs):
    """Given question pairs, conevrting to list of InputExample"""
    examples = []
    for (i, qpair) in enumerate(sentence_pairs):
      guid = "predict-%d" % (i)
      # converting questions to utf-8 and creating InputExamples
      text_a = tokenization.convert_to_unicode(qpair[3])
      text_b = tokenization.convert_to_unicode(qpair[0])
      label = int(qpair[2])
      # We will add label  as 0, because None is not supported in converting to features
      examples.append(
          run_classifier.InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
    return examples

  def get_predict_examples(self, sentence_pairs):
    """Given question pairs, conevrting to list of InputExample"""
    examples = []
    for (i, qpair) in enumerate(sentence_pairs):
      guid = "predict-%d" % (i)
      # converting questions to utf-8 and creating InputExamples
      text_a = tokenization.convert_to_unicode(qpair[3])
      text_b = tokenization.convert_to_unicode(qpair[0])
      # We will add label  as 0, because None is not supported in converting to features
      examples.append(
          run_classifier.InputExample(guid=guid, text_a=text_a, text_b=text_b, label=0))
    return examples
  
  def _create_examples(self, lines, set_type):
    """Creates examples for the training, dev and test sets."""
    examples = []
    for (i, line) in lines.iterrows():
      guid = "%s-%d" % (set_type, i)
      if set_type=='test':
        # removing header and invalid data
        if i == 0 or len(line)!=4:
          print(guid, line)
          continue
        text_a = tokenization.convert_to_unicode(line[3])
        text_b = tokenization.convert_to_unicode(line[0])
        label = int(line[2]) # We will use zero for test as convert_example_to_features doesn't support None
      else:
        # removing header and invalid data
        if i == 0 or len(line)!=4:
          print(guid, line)
          continue
        text_a = tokenization.convert_to_unicode(line[3])
        text_b = tokenization.convert_to_unicode(line[0])
        label = int(line[2])
      examples.append(
          run_classifier.InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
    return examples

  def get_labels(self):
    "return class labels"
    return [0,1]

Also we initilize our hyperprams, prepare the training data and initialize TPU config.

In [18]:
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
PREDICT_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 5.0
MAX_SEQ_LENGTH = 128
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500

processors = {
  "cola": run_classifier.ColaProcessor,
  "jokes":JokeProcessor,
  "mnli": run_classifier.MnliProcessor,
  "mrpc": run_classifier.MrpcProcessor,
}
processor = processors[TASK.lower()]()
label_list = processor.get_labels()
if TASK == "Jokes":
  train=reddit.sample(frac=0.8,random_state=200) #random state is a seed value
  test=reddit.drop(train.index)
  print("length of train set ",len(train),"length of test set ",len(test))
  #train_test_examples = train_test_split(reddit, test_size=0.2)
  TASK_DATA_DIR =[train,test]
# Compute number of train and warmup steps from batch size
train_examples = processor.get_train_examples(TASK_DATA_DIR)
print("length of train examples set",len(train_examples))
num_train_steps = int(len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Setup TPU related config
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
NUM_TPU_CORES = 8
ITERATIONS_PER_LOOP = 1000

def get_run_config(output_dir):
  return tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=output_dir,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))


length of train set  79037 length of test set  19759
train-0 body                                       Addadictomy.
id                                               3un5n5
score                                                 0
title    What is called when a woman gets a sex change?
Name: 0, dtype: object
length of train examples set 79036


# Fine-tune and Run Predictions on a pretrained BERT Model from TF Hub

This section demonstrates fine-tuning from a pre-trained BERT TF Hub module and running predictions.


In [19]:
# Force TF Hub writes to the GS bucket we provide.
os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR

model_fn = run_classifier_with_tfhub.model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  bert_hub_module_handle=BERT_MODEL_HUB
)

estimator_from_tfhub = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)


INFO:tensorflow:Using config: {'_model_dir': 'gs://humorbot3000/hum0rb0t/3000/Jokes', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.6.80.130:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fcb6fb87da0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.6.80.130:8470', '_evaluation_master': 'grpc://10.6.80.130:8470', '_is_chief': True, '_num_ps_repli

INFO:tensorflow:Using config: {'_model_dir': 'gs://humorbot3000/hum0rb0t/3000/Jokes', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.6.80.130:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fcb6fb87da0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.6.80.130:8470', '_evaluation_master': 'grpc://10.6.80.130:8470', '_is_chief': True, '_num_ps_repli

INFO:tensorflow:_TPUContext: eval_on_tpu True


INFO:tensorflow:_TPUContext: eval_on_tpu True


At this point, you can now fine-tune the model, evaluate it, and run predictions on it.

In [0]:
# Train the model
def model_train(estimator):
  print('MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...')
  # We'll set sequences to be at most 128 tokens long.
  train_features = run_classifier.convert_examples_to_features(
      train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started training at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(train_examples)))
  print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
  tf.logging.info("  Num steps = %d", num_train_steps)
  train_input_fn = run_classifier.input_fn_builder(
      features=train_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=True)
  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print('***** Finished training at {} *****'.format(datetime.datetime.now()))



In [21]:
# Train the model
def model_train(estimator):
  print('MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...')
  # We'll set sequences to be at most 128 tokens long.
  train_features = run_classifier.convert_examples_to_features(
      train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started training at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(train_examples)))
  print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
  tf.logging.info("  Num steps = %d", num_train_steps)
  train_input_fn = run_classifier.input_fn_builder(
      features=train_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=True)
  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print('***** Finished training at {} *****'.format(datetime.datetime.now()))

estimator_from_tfhub = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)


INFO:tensorflow:Using config: {'_model_dir': 'gs://humorbot3000/hum0rb0t/3000/Jokes', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.6.80.130:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fcb6e8ed3c8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.6.80.130:8470', '_evaluation_master': 'grpc://10.6.80.130:8470', '_is_chief': True, '_num_ps_repli

INFO:tensorflow:Using config: {'_model_dir': 'gs://humorbot3000/hum0rb0t/3000/Jokes', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.6.80.130:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fcb6e8ed3c8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.6.80.130:8470', '_evaluation_master': 'grpc://10.6.80.130:8470', '_is_chief': True, '_num_ps_repli

INFO:tensorflow:_TPUContext: eval_on_tpu True


INFO:tensorflow:_TPUContext: eval_on_tpu True


In [0]:
model_train(estimator_from_tfhub)

MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...



INFO:tensorflow:Writing example 0 of 79036


INFO:tensorflow:Writing example 0 of 79036


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


...

Instructions for updating:
Use standard file utilities to get mtimes.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [0]:
def model_eval(estimator):
  # Eval the model.
  eval_examples = processor.get_dev_examples(TASK_DATA_DIR)
  eval_features = run_classifier.convert_examples_to_features(
      eval_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started evaluation at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(eval_examples)))
  print('  Batch size = {}'.format(EVAL_BATCH_SIZE))

  # Eval will be slightly WRONG on the TPU because it will truncate
  # the last batch.
  eval_steps = int(len(eval_examples) / EVAL_BATCH_SIZE)
  eval_input_fn = run_classifier.input_fn_builder(
      features=eval_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=False,
      drop_remainder=True)
  result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
  print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
  output_eval_file = os.path.join(OUTPUT_DIR, "eval_results.txt")
  with tf.gfile.GFile(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    for key in sorted(result.keys()):
      print('  {} = {}'.format(key, str(result[key])))
      writer.write("%s = %s\n" % (key, str(result[key])))


In [0]:
model_eval(estimator_from_tfhub)

## Prediction and Analysis:


In [0]:
examples=[[ 'They played it on my flight home and there were only two walkouts.','4','1',
"I don't know why everyone is saying Cats (the movie) was bad."]]
def model_predict_from(estimator,examples):
  # Make predictions on a subset of eval examples
  prediction_examples = processor.convert_examples(examples)
  input_features = run_classifier.convert_examples_to_features(prediction_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=True)
  predictions = estimator.predict(predict_input_fn)
  rows=[]
  for example, prediction in zip(prediction_examples, predictions):
    row={'q1': str(example.text_a),'q2': str(example.text_b),'lbl': str(example.label),'pred': str(prediction['probabilities'])}
    rows.append(row)
  return rows 
def model_predict(estimator):
  # Make predictions on a subset of eval examples
  prediction_examples = processor.get_dev_examples(TASK_DATA_DIR)
  input_features = run_classifier.convert_examples_to_features(prediction_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=True)
  predictions = estimator.predict(predict_input_fn)
  rows=[]
  for example, prediction in zip(prediction_examples, predictions):
    row={'q1': str(example.text_a),'q2': str(example.text_b),'lbl': str(example.label),'pred': str(prediction['probabilities'])}
    rows.append(row)
  return rows 


In [0]:
data2 = model_predict_from(estimator_from_tfhub,examples)
data2

In [0]:
data=model_predict(estimator)
SUB_TASK='jokes'
postprocessing(data,[])

In [0]:
data

In [0]:
'''current_tasks=["QQP"]
current_tasks=["StackExchange",'StackCola']
predict_results=[]
predict_setups=[]
for SUB_TASK in current_tasks:
  !gsutil -m cp gs://{BUCKET}/Final_Data/{SUB_TASK}/dev.tsv /content/Final_Data/{SUB_TASK}/dev.tsv
  
  TASK_PREDICT="QQP" if SUB_TASK in ["QQP",'StackExchange','StackCola'] else "PAWS"
  TASK_PREDICT_DIR = 'Final_Data/'+SUB_TASK+'/'
  #TASK_PREDICT_DIR='/content'
  processor = processors[TASK_PREDICT.lower()]()
  label_list = processor.get_labels()
  predict_result =model_predict(estimator_from_tfhub,TASK_PREDICT_DIR,processor)
  predict_setup={'TASK_PREDICT':TASK_PREDICT+"_"+SUB_TASK}
  predict_results.append(predict_result)
  predict_setups.append(predict_setup)'''

In [0]:
def postprocessing(data,predict_set_up):
  df = pd.DataFrame(data)
  df['pred_lbl']=0
  df['type_error']="None"

  df['lbl']=df['lbl'].astype(int)
  preds=[]
  for table in df.pred.values:

    pred_row=json.loads(table.replace(" ",',',1))
    preds.append(pred_row)
  preds=np.array(preds)
  print(preds.shape)
  df['first']=preds[:,0].astype(float)
  df['second']=preds[:,1].astype(float)
  df.loc[ ((df['first'])< (df['second'])),['pred_lbl']]= 1
  df.loc[((df['lbl']==1) & (df['pred_lbl']==1)),['type_error']]="TP" 
  df.loc[((df['lbl']==0) & (df['pred_lbl']==0)),['type_error']]="TN" 
  df.loc[((df['lbl']==0) & (df['pred_lbl']==1)),['type_error']]="FP" 
  df.loc[((df['lbl']==1) & (df['pred_lbl']==0)),['type_error']]="FN" 
  filename="prediction_"+BERT_MODEL+TASK+"_"+SUB_TASK+".csv"
  df.to_csv(filename)
  !gsutil  -m cp /content/{filename} gs://{BUCKET}/prediction_results/{filename}
  error_acronyms=['TN','TP','FN','FP']
  met_dict={error_type:df[df['type_error']==error_type] for error_type in error_acronyms}
  met={error_type:met_dict[error_type].shape[0] for error_type in error_acronyms}

  #Precision = TP/TP+FP
  precision=met['TP']/(met['TP']+met['FP'])
  #Recall = TP/TP+FN
  recall=met['TP']/(met['TP']+met['FN'])
  #F1 Score = 2*(Recall * Precision) / (Recall + Precision)
  f1_score=2*(recall * precision) / (recall + precision)


  accuracy=(met['TP']+met['TN'])/sum([met[m] for m in error_acronyms])
  score_names=('accuracy','precision','recall', 'f1_score')
  sc=(accuracy,precision,recall, f1_score)
  scores={}
  for name,score in zip(score_names,sc):
    print(name, score)
    scores={name:score}
  out={**predict_set_up,**scores,**met}

  e_a=['TN','TP','FN','FP']
  errors=['True Negatives','True positives','False Negatives','False positives']
  for i, error_type in zip(e_a,errors):
    print('The number of '+error_type+' is: ',met_dict[i].shape[0])
  print('using ',TASK, SUB_TASK)
  print('Analysis of ',TASK, SUB_TASK)
  print('Examples of False positives:')
  FP=met_dict['FP'][['q1','q2']].head(15).to_numpy()
  FN=met_dict['FN'][['q1','q2']].head(15).to_numpy()
  for q in range(1,10):
    if q<=FP.shape[0]:
      print('Example: ',q)
      print('Statement 1:',FP[q,0])
      print('Statement 2:',FP[q,1])

  print()  
  print('Examples of False Negatives:')
  for q in range(1,10):
    if q<=FN.shape[0]:
      print('Example: ',q)
      print('Statement 1:',FN[q,0])
      print('Statement 2:',FN[q,1])
      print()
  return out

In [0]:
postprocessing(predict_results[0],predict_setups[0])

In [0]:
import numpy as np

import json
import pandas as pd

df = pd.DataFrame(data)
df['pred_lbl']=0
df['type_error']="None"

df['lbl']=df['lbl'].astype(int)
preds=[]
for table in df.pred.values:

  pred_row=json.loads(table.replace(" ",',',1))
  preds.append(pred_row)
preds=np.array(preds)
print(preds.shape)
df['first']=preds[:,0].astype(float)
df['second']=preds[:,1].astype(float)
df.loc[ ((df['first'])< (df['second'])),['pred_lbl']]= 1
df.loc[((df['lbl']==1) & (df['pred_lbl']==1)),['type_error']]="TP" 
df.loc[((df['lbl']==0) & (df['pred_lbl']==0)),['type_error']]="TN" 
df.loc[((df['lbl']==0) & (df['pred_lbl']==1)),['type_error']]="FP" 
df.loc[((df['lbl']==1) & (df['pred_lbl']==0)),['type_error']]="FN" 
filename="prediction_"+BERT_MODEL+"_label_wiki.csv"
df.to_csv(filename)
!gsutil  -m cp /content/{filename} gs://{BUCKET}/prediction_results/{filename}

In [0]:
FN=df[df['type_error']=="FN"]
FP=df[df['type_error']=="FP"]
print('The number of False positives is: ',FP[['q1','q2']].shape[0])
print('The number of False Negatives is: ',FN[['q1','q2']].shape[0])

print('Analysis of PAWS WIKI')
print('Examples of False positives:')
FP=FP[['q1','q2']].head(15).to_numpy()
FN=FN[['q1','q2']].head(15).to_numpy()
for q in range(1,10):
  print('Example: ',q)
  print('Statement 1:',FP[q,0])
  print('Statement 2:',FP[q,1])

print()  
print('Examples of False Negatives:')
for q in range(1,10):
  print('Example: ',q)
  print('Statement 1:',FN[q,0])
  print('Statement 2:',FN[q,1])
  print()

# Fine-tune and run predictions on a pre-trained BERT model from checkpoints

Alternatively, you can also load pre-trained BERT models from saved checkpoints.

In [0]:
# Setup task specific model and TPU running config.
BERT_PRETRAINED_DIR = 'gs://cloud-tpu-checkpoints/bert/' + BERT_MODEL 
print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))
!gsutil ls $BERT_PRETRAINED_DIR

CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')

model_fn = run_classifier.model_fn_builder(
  bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
  num_labels=len(label_list),
  init_checkpoint=INIT_CHECKPOINT,
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  use_one_hot_embeddings=True
)

OUTPUT_DIR = OUTPUT_DIR.replace('bert-tfhub', 'bert-checkpoints')
tf.gfile.MakeDirs(OUTPUT_DIR)

estimator_from_checkpoints = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)

Now, you can repeat the training, evaluation, and prediction steps.

In [0]:
model_train(estimator_from_checkpoints)

In [0]:
model_eval(estimator_from_checkpoints)

In [0]:
model_predict(estimator_from_checkpoints)

## What's next

* Learn about [Cloud TPUs](https://cloud.google.com/tpu/docs) that Google designed and optimized specifically to speed up and scale up ML workloads for training and inference and to enable ML engineers and researchers to iterate more quickly.
* Explore the range of [Cloud TPU tutorials and Colabs](https://cloud.google.com/tpu/docs/tutorials) to find other examples that can be used when implementing your ML project.

On Google Cloud Platform, in addition to GPUs and TPUs available on pre-configured [deep learning VMs](https://cloud.google.com/deep-learning-vm/),  you will find [AutoML](https://cloud.google.com/automl/)*(beta)* for training custom models without writing code and [Cloud ML Engine](https://cloud.google.com/ml-engine/docs/) which will allows you to run parallel trainings and hyperparameter tuning of your custom models on powerful distributed hardware.
